In [1]:
##

import gym

from stable_baselines import GAIL, SAC
from stable_baselines.gail import ExpertDataset,ExpertDatasetConsecutive, generate_expert_traj

import gym
import numpy as np

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2
from stable_baselines import PPO1
# from particle_env import PrticleEnv
from particle_env_continuous_closer import PrticleEnv

import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("%s" % (info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())





/data3/maze/project/stable-baselines/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data3/maze/project/stable-baselines/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data3/maze/project/stable-baselines/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data3/maze/project/stabl

In [2]:
# Generate expert trajectories (train expert)
env = PrticleEnv(alpha=1,beta=10,win_thre=1, max_timestep=256,for_circle_traj=True)

model = PPO1.load("model/part_circle_exp1_epoch20.zip")
model.set_env(env)
generate_expert_traj(model, 'expert_part_circle_exp1_epoch20', n_timesteps=100, n_episodes=10)



episode_returns (10,)
obs (519, 4)
actions (519, 2)
episode_starts (519,)
rewards (519,)


{'actions': array([[-5.7473974, 10.       ],
        [-3.9359896, 10.       ],
        [-6.6416683, 10.       ],
        ...,
        [10.       ,  6.5356855],
        [10.       , 10.       ],
        [ 9.876711 ,  7.311096 ]], dtype=float32),
 'episode_returns': array([224.19479492, 332.84827169, 330.3090893 , 253.27759398,
        355.7823128 , 267.04124183, 190.00414597, 307.39433897,
        319.23175224, 266.77246536]),
 'episode_starts': array([ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False,  True, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False

In [ ]:
# Load the expert dataset
dataset = ExpertDataset(expert_path='expert_part_circle_exp1_epoch20.npz', traj_limitation=10, verbose=1)

model = GAIL('MlpPolicy'\
             ,DummyVecEnv([lambda: PrticleEnv(alpha=1,beta=10,win_thre=1, max_timestep=256,for_circle_traj=True)])\
             , dataset, verbose=1)
# Note: in practice, you need to train for 1M steps to have a working policy
model.learn(total_timesteps=int(1e6))
model.save("gail_part_circle_exp1_epoch20")

del model # remove to demonstrate saving and loading



In [3]:
from stable_baselines.gail.gail_useTrueReward_consecutiveTimeStep import GAIL

model = GAIL.load("model/gail_exp3_300k.zip")

env = PrticleEnv(alpha=1,beta=100,win_thre=1, max_timestep=256)

# Enjoy trained agent
dones = True
# for _ in range(1000):
while True:
    if dones:
        obs = env.reset()
    action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)
    obs, rewards, dones, info = env.step(env.action_space.sample())
    obs = np.round(obs,2)
#     show_state(env,info="step:%4d reward:%.2fx%.2f=%.2f" %(env.curr_timestep,env.shift_reward,env.velocity_reward,rewards))
    show_state(env,info="step:%4d reward:%.2f" %(env.curr_timestep,rewards))

env.close()

NameError: name 'base' is not defined

<Figure size 432x288 with 0 Axes>